<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/6%20-%20Sentiment%20Analysis/sentiment_analysis_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis con NLTK e Vader
In questo notebook vedremo come estrarre il sentiment di una frase, che ci permette di comprendere se l'emozione espressa è positiva o negativa, utilizzando il modulo Vader con NLTK

## Vader
Vader (Valene Aware Dictionary for Sentiment Reasoning) è un modulo utilizzato per la sentiment analysis, specialmente sui social media, che contiene un dizionario di parole già annotate come positive e negative che, in combinazione con un set di regole lessicali, ci permette di sapere non solo se una frase contiene un sentiment positivo o negativo, ma anche quanto è positivo o negativo tale sentiment.
<br>
Per usare Vade con NLTK dobbiamo scaricare e installare il modulo 'vader_lexicon'.

In [0]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

Ora importiamo la classe *SentimentIntensityAnalyzer*, creiamo un'alias per abbreviare un po' il nome. Possiamo ottenere il sentiment di una frase usando il metodo *polarity_score(text)*, proviamo con una semplice recensione di una app.

In [76]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SentimentAnalyzer

sentiment_analyzer = SentimentAnalyzer()

text = "I love this app" # Amo questa app
sentiment_analyzer.polarity_scores(text)

{'compound': 0.6369, 'neg': 0.0, 'neu': 0.323, 'pos': 0.677}

L'ouput è un dizionario che contiene 4 valori:
 - **compound**: è il sentiment tra -1 e 1, maggiore di zero è positivo, minore è negativo.
 - **neg**: punteggio per le parole negative, tra 0 e 1.
 - **neu**: punteggio per le parole neutre, tra 0 e 1.
 - **pos**: punteggio per le parole positive, tra 0 e 1.
 
 
 In questo caso la recensione è molto positiva. Proviamo con una negativa.

In [77]:
text = "I didn't like this app, unistalled" # Non mi è piaciuta questa app, disinstallata.
sentiment_analyzer.polarity_scores(text)

{'compound': -0.2755, 'neg': 0.345, 'neu': 0.655, 'pos': 0.0}

La recensione viene riconosciuta come piuttosto negativa, nota come l'algoritmo è riuscito a comprendere la negazione "didn't like", se avesse ragionato parola per parola il sentiment sarebbe stato positivo dato che like è una parola positiva.
<br>
Proviamo con una recensione un po' più ambigua.

In [79]:
text = "I like this app even if the user interface is poor" # Mi piace questa app anche se l'interfaccia grafica è un po' povera.
sentiment_analyzer.polarity_scores(text)

{'compound': -0.1531, 'neg': 0.228, 'neu': 0.588, 'pos': 0.184}

La recensione viene riconosciuta come lievemente negativa, cambiamo 'like' con 'love'.

In [80]:
text = "I love this app even if the user interface is poor" # Amo questa app anche se l'interfaccia grafica è un po' povera.
sentiment_analyzer.polarity_scores(text)

{'compound': 0.2732, 'neg': 0.203, 'neu': 0.523, 'pos': 0.275}

In questo caso la recensione diventa positiva, le parole scritte in maiuscolo hanno un'intensità maggiore (nel web il maiuscolo equivale ad urlare HAI CAPITO ?)

In [81]:
text = "I LOVE this app even if the user interface is poor" # AMO questa app anche se l'interfaccia grafica è un po' povera.
sentiment_analyzer.polarity_scores(text)

{'compound': 0.4278, 'neg': 0.193, 'neu': 0.499, 'pos': 0.308}

La recensione è diventata ancora più positiva, anche un utilizzo smodato della punteggiatura influisce sul sentiment, dato che può manifestare eccitazione.

In [82]:
text = "I LOVE this app !!! Eeven if the user interface is poor" # AMO questa app !!! Anche se l'interfaccia grafica è un po' povera.
sentiment_analyzer.polarity_scores(text)

{'compound': 0.5732, 'neg': 0.173, 'neu': 0.503, 'pos': 0.324}

## Sentiment su recensioni reali
Vediamo di fare adesso una prova più impegnativa utilizzando un dataset di 12 recensioni reali lasciate per la mia skill Alexa [hi-fit](https://www.amazon.com/Amanity-HiFit-Home-Workouts/dp/B06XQMFVHQ) nello store americano. Importiamo il dataset all'interno di un Dataframe di Pandas, le colonne del CSV sono serarate da un punto e virgola e non da una virgola (dato che le frasi delle recensioni contengono virgole), passiamo questa informazione a pandas tramite il parametro *sep*.

In [84]:
import pandas as pd

reviews_url = "https://raw.githubusercontent.com/ProfAI/nlp00/master/6%20-%20Sentiment%20Analysis/data/hifit_reviews.csv"
reviews = pd.read_csv(reviews_url, sep=";")
reviews

Username                            title  \
0            Amazon Customer         Great With Modifications   
1              Janey_monster  Won't take 'Yes' for an answer!   
2               Gary Mercado  Won't respond to my yes command   
3            Amazon Customer                        Excellent   
4   Linda Charlton-Gunderson       Does not always hear ready   
5             Mallory galvin                         Terrible   
6            Amazon Customer                      Challenging   
7                 jenisse210      Great idea but doesn't work   
8                     Matteo                     Fantastic!!!   
9            Amazon Customer                       Five Stars   
10                     aisha      Great but needs improvement   
11           Rebekah Bertram                          love it   

                                               review  rating  
0   The recovery times are longer as some have men...     4.5  
1   Just like another reviewer said. Between exerc...     2.0  
2   The skill won't respond to the yes command and...     2.0  
3   So far I’m happy of this new Skill. I don’t ha...     5.0  
4   I said ready and she would shut off. But when ...     2.0  
5   Slow and annoying. I don't need 30 seconds to ...     1.0  
6   I love bodyweight workouts and they look well-...     5.0  
7   I've tried to start twice now and I get to sta...     2.0  
8   I've never seen something like that, I am a bi...     5.0  
9   I tried the app and it's the best one for my d...     5.0  
10  Great app if it just don't shut itself off tim...     4.0  
11  my kids do this woth me. i kove it. great musi...     5.0

Come vedi abbiamo nome utente, titolo della recensione, testo della recensione e il rating da 1 a 5. Sia il titolo che il testo contengono informazioni utili per la sentiment analysis, quindi uniamoli in un unica stringa.

In [59]:
full_review = reviews["title"]+" "+reviews["review"]

print(type(full_review))

print(full_review.loc[0])

<class 'pandas.core.series.Series'>
Great With Modifications The recovery times are longer as some have mentioned, but I like to do active recoveries and mix in some cardio, so I jog, do jumping jacks, high knees, etc. during the recovery. Since my workouts are usually 50 minutes, and this is a backup, I will modify the different exercises on different rounds as well. For example, do static lunge squats or backwards lunges instead of just lunges. This makes the workout more challenging and keeps me from boredom. The only thing I don’t like is you can’t pick your music. Overall, this app is just like any workout - it’s yours, do what you want with it.


Adesso abbiamo una Series che contiene la review per intero (titolo+corpo), utilizziamo il metodo apply per passare ogni colonna al metodo *polarity_scores(text)*.

In [62]:
sentiments = full_review.apply(lambda review: sentiment_analyzer.polarity_scores(review))
sentiments.head()

0    {'neg': 0.024, 'neu': 0.794, 'pos': 0.182, 'co...
1    {'neg': 0.03, 'neu': 0.831, 'pos': 0.139, 'com...
2    {'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...
3    {'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compou...
4    {'neg': 0.084, 'neu': 0.771, 'pos': 0.144, 'co...
dtype: object

Il risultato è un'altra Series, in cui ogni colonna contiene il dizionario ritornato dal metodo *polarity_scores(text)*.

In [85]:
print(type(sentiments.iloc[0]))
sentiments.iloc[0]

<class 'dict'>


{'compound': 0.9541, 'neg': 0.024, 'neu': 0.794, 'pos': 0.182}

Creiamo una nuova colonna 'sentiment score' all'interno del DataFrame, all'interno della quale salviamo il valore del sentiment generale della frase.

In [87]:
reviews['sentiment score'] = sentiments.apply(lambda sentiment: sentiment["compound"])
reviews

Username                            title  \
0            Amazon Customer         Great With Modifications   
1              Janey_monster  Won't take 'Yes' for an answer!   
2               Gary Mercado  Won't respond to my yes command   
3            Amazon Customer                        Excellent   
4   Linda Charlton-Gunderson       Does not always hear ready   
5             Mallory galvin                         Terrible   
6            Amazon Customer                      Challenging   
7                 jenisse210      Great idea but doesn't work   
8                     Matteo                     Fantastic!!!   
9            Amazon Customer                       Five Stars   
10                     aisha      Great but needs improvement   
11           Rebekah Bertram                          love it   

                                               review  rating  sentiment score  
0   The recovery times are longer as some have men...     4.5           0.9541  
1   Just like another reviewer said. Between exerc...     2.0           0.8763  
2   The skill won't respond to the yes command and...     2.0           0.8591  
3   So far I’m happy of this new Skill. I don’t ha...     5.0           0.7566  
4   I said ready and she would shut off. But when ...     2.0           0.6531  
5   Slow and annoying. I don't need 30 seconds to ...     1.0          -0.7003  
6   I love bodyweight workouts and they look well-...     5.0           0.7964  
7   I've tried to start twice now and I get to sta...     2.0           0.8139  
8   I've never seen something like that, I am a bi...     5.0           0.9130  
9   I tried the app and it's the best one for my d...     5.0           0.7830  
10  Great app if it just don't shut itself off tim...     4.0           0.8074  
11  my kids do this woth me. i kove it. great musi...     5.0           0.8622

Se vogliamo aggiungiamo anche un label per interpretare il sentiment, definiamo una funzione che mappa il valore del sentimento a un label.

In [0]:
def sentiment_to_label(sentiment_value):
    
    if(sentiment_value <= -0.75):
      return "fortemente negativa"
    elif(sentiment_value <= -0.5):
      return "molto negativa"
    elif(sentiment_value <= -0.25):
      return "piuttosto negativa"
    elif(sentiment_value <=0):
      return "negativa"
    elif(sentiment_value<=0.25):
      return "piuttosto positiva"
    elif(sentiment_value<=0.5):
      return "positiva"
    elif(sentiment_value<=0.75):
      return "molto positiva"
    elif(sentiment_value>0.75):
      return "fortemente positiva"
    
    return "errore"

Ora utilizziamo nuovamente il metodo *.apply* per creare la nuova colonna 'sentiment label'.

In [73]:
reviews['sentiment label'] = reviews['sentiment score'].apply(lambda sentiment: sentiment_to_label(sentiment))
reviews.head()

Username                            title  \
0            Amazon Customer         Great With Modifications   
1              Janey_monster  Won't take 'Yes' for an answer!   
2               Gary Mercado  Won't respond to my yes command   
3            Amazon Customer                        Excellent   
4   Linda Charlton-Gunderson       Does not always hear ready   
5             Mallory galvin                         Terrible   
6            Amazon Customer                      Challenging   
7                 jenisse210      Great idea but doesn't work   
8                     Matteo                     Fantastic!!!   
9            Amazon Customer                       Five Stars   
10                     aisha      Great but needs improvement   
11           Rebekah Bertram                          love it   

                                               review  rating  \
0   The recovery times are longer as some have men...     4.5   
1   Just like another reviewer said. Between exerc...     2.0   
2   The skill won't respond to the yes command and...     2.0   
3   So far I’m happy of this new Skill. I don’t ha...     5.0   
4   I said ready and she would shut off. But when ...     2.0   
5   Slow and annoying. I don't need 30 seconds to ...     1.0   
6   I love bodyweight workouts and they look well-...     5.0   
7   I've tried to start twice now and I get to sta...     2.0   
8   I've never seen something like that, I am a bi...     5.0   
9   I tried the app and it's the best one for my d...     5.0   
10  Great app if it just don't shut itself off tim...     4.0   
11  my kids do this woth me. i kove it. great musi...     5.0   

    sentiment score      sentiment label  
0            0.9541  fortemente positiva  
1            0.8763  fortemente positiva  
2            0.8591  fortemente positiva  
3            0.7566  fortemente positiva  
4            0.6531       molto positiva  
5           -0.7003       molto negativa  
6            0.7964  fortemente positiva  
7            0.8139  fortemente positiva  
8            0.9130  fortemente positiva  
9            0.7830  fortemente positiva  
10           0.8074  fortemente positiva  
11           0.8622  fortemente positiva

Tutte le recensioni positive sono state classificate con la giusta intensità, mentre per alcune negative la sentiment analysis è stata eseguita in maniera un po' troppo ottimistica. Vediamo una di queste recensioni.

In [88]:
reviews.loc[1].review

'Just like another reviewer said. Between exercises, when it asks if your ready to start, I say yes and it says "okay I\'ll keep that in mind for next time.... Blah blah blah..." and closes me out before I finish my set. If not for that, and maybe also not knowing what a \'burpee\' is (perhaps I\'m a bit behind on exercise vocabulary... But a description option would be nice!), I\'d rate 5. Please fix?? (P. S. I have an echo dot 2nd gen)'

La recensione riporta:
<br><br>
*'Proprio come ha detto un altro recensore. Tra un esercizio e l'altro, quando ti chiede se sei pronto per iniziare, io dico di sì e dice "okay, tienilo a mente per la prossima volta .... Blah blah blah ..." e mi chiude prima che finisca il mio esercizio. Se non fosse per quello, e forse anche non sapendo cosa sia un "burpee" (forse sono un po 'indietro sul vocabolario degli esercizi ... Ma una opzione di descrizione sarebbe bella!), Il mio voto è 5. Per favore risolvete ?? (P. S. Ho un Echo  Dot di 2a generazione) '*
<br><br>
Come vedi non sono presenti parole che lascino intendere un sentiment negativo, mentre ce ne sono diverse positive (bella, il mio voto è 5, okay).

## Link utili

- http://www.nltk.org/howto/sentiment.html